In [29]:
import advent
advent.scrape(2016, 22)
data = advent.get_lines(22)[2:]
lines = []
for line in data: # [data[0], data[488]]:
    lines.append((
        int(line[16:18].replace('-', '')),
        int(line[19:21].replace('y', '')),
        int(line[23:27]),
        int(line[29:33]),
        int(line[36:40]),

    ))
# x, y, total, used, free

def viable(a, b):
    if a[3] == 0: return False
    if a[0] == b[0] and a[1] == b[1]: return False
    return a[3] <= b[4]

result = 0
for i in range(len(lines)):
    for j in range(len(lines)):
        #print(i, j, viable(lines[i], lines[j]))
        if viable(lines[i], lines[j]):
            result += 1
print(result)


976


In [ ]:
from typing import Iterable
from advent.maze import solve_maze
# Well well well.
# Let's try the usual brute force BFS/dijkstra storing the whole board as a node
# The board itself is 37x27 and each position contains 3 numbers: used, avail, payload

start_r = [(line[3], line[4], 0) for line in lines]
start_r[972] = (start_r[972][0], start_r[972][1], 1)
# payload data. 972 i just manually looked up but it's actually just (37-1) * 27

# Now the trick (because I tried it without and it took too long):
# There are only three types of nodes: normal (1, 0, x), empty (0, 1, x) and big (2, 0, x)
start_s = []
for s in start_r:
    if s[1] > 50: start_s.append((0, 1, s[2])) # empty
    elif s[0] > 100: start_s.append((2, 0, s[2])) # big 
    else: start_s.append((1, 0, s[2])) # normal

Board = tuple[tuple[tuple[int, int, int], ...], ...]
start: Board = tuple([
    tuple(start_s[i:(i+27)])
    for i in range(0, len(lines), 27)
])


In [99]:
tadd = lambda x, y: (x[0] + y[0], x[1] + y[1])

def viable(a: tuple[int, int, int], b:tuple[int, int, int]):
    return a[0] > 0 and a[0] <= b[1]

def move(a: tuple[int, int, int], b:tuple[int, int, int]):
    # Assumes its viable
    return (0, a[0] + a[1], 0), (a[0] + b[0], b[1] - a[0], a[2] + b[2])

def adj_coords(coord: tuple[int, int]) -> Iterable[tuple[int, int]]:
    offsets = [(0, 1), (0, -1), (1, 0), (-1, 0)]
    for offset in offsets:
        new = tadd(coord, offset)
        if new[0] > 0 and new[0] < 37 and new[1] > 0 and new[1] < 27:
            yield new

def adjacent(board: Board) -> Iterable[tuple[Board, int]]:
    for i in range(37):
        for j in range(27):
            for coord in adj_coords((i, j)):
                k, l = coord
                if viable(board[i][j], board[k][l]):
                    a, b = move(board[i][j], board[k][l])
                    newboard = list([list(board[m]) for m in range(37)])
                    newboard[i][j] = a
                    newboard[k][l] = b
                    #print(i, j, k, l)
                    yield tuple([tuple(newboard[m]) for m in range(37)]), 1

def is_target(board: Board):
    return board[0][0][2] == 1


In [101]:
# What makes this so hard is that due to the size it's basicaly impossible to check if adjacent is correct
r = solve_maze(start, is_target, adjacent)
print(r[0])

  0%|          | 0/1 [00:00<?, ?it/s]

40
